• Reading from video file and camera
• Simple Color Detection and Tracking
• Point Operator
• Adjusting Contrast and Brightness
• Array Slicing with NumPy
• Image Blending

In [9]:
import cv2
import time
import numpy as np
import argparse
import matplotlib.pyplot as plt

In [2]:
# cv2.waitKey(0) for displaying image, and cv2.waitKey(1) for displaying video

# create a video capture object from a file
video = cv2.VideoCapture('Video/City.mp4')

if video.isOpened() == False:
    print('Error opening video stream or file')
else:
    fps = video.get(cv2.CAP_PROP_FPS) # frame rate info
    print('FPS: ', fps, 'FPS')
    frame_Count = video.get(cv2.CAP_PROP_FRAME_COUNT) # get frame count
    print('Frame Count: ', frame_Count)

    cv2.namedWindow("Frame")
    cv2.setWindowProperty("Frame",cv2.WND_PROP_TOPMOST, 1)

    while video.isOpened():
        ret, frame = video.read() # capture next frame
        if ret == True:
            cv2.imshow("Frame", frame)
            time.sleep(1/fps) # adjust frame rate
            if cv2.waitKey(10000) & 0xFF == ord('x'): # wait to press ESC
                break
            else:
                break # no next frame

video.release()
cv2.destroyAllWindows()

FPS:  29.9788285109386 FPS
Frame Count:  354.0


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sudhir/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [34]:
# create video capture from a camera

cam = cv2.VideoCapture(0) # get camera device
if cam.isOpened() == True:

    print("Camera is Open")

    cv2.namedWindow("Camera")
    cv2.setWindowProperty("Camera", cv2.WND_PROP_TOPMOST, 1)

    while cam.isOpened():
        ret, frame = cam.read() # capture the next frame

        if ret == True:
            cv2.imshow("Camera", frame) # show the frame
            if cv2.waitKey(50000) == ord('x'):
                break
        else:
            break

cam.release()
cv2.destroyAllWindows()
print("Camera is closed")

Camera is Open
Camera is closed


In [32]:
# color detection in Image

rgb = cv2.imread('Video/rgb.jpg')
hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)

# define range of color
lower_red = np.array([0, 200, 20]) # min h, s, v
upper_red = np.array([10, 255, 255]) # max h, s, v

# create mask for red color
red_mask = cv2.inRange(hsv, lower_red, upper_red)

# contour in mask
contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

"""contours: It is a Python list that contains all the detected contours. 
Each contour is represented as a numpy array of shape (n, 1, 2),
where n is the number of points on the contour. Each point contains the (x, y) coordinates of the contour."""

'''
hierarchy: It is a representation of the hierarchical relationships between contours. 
It provides information about the topology of the contours, such as which contours are nested or have holes.
 The hierarchy is represented as a numpy array of shape (1, N, 4), where N is the total number of contours. 
 Each contour's hierarchy is defined by four values: [Next, Previous, First Child, Parent].

Next: It represents the index of the next contour at the same hierarchical level. If there is no next contour, the value is -1.

Previous: It represents the index of the previous contour at the same hierarchical level. If there is no previous contour, the value is -1.

First Child: It represents the index of the first child contour. If there is no child contour, the value is -1.

Parent: It represents the index of the parent contour. If there is no parent contour, the value is -1.
'''


# create output image and draw contour in green
output1 = rgb.copy()
cv2.drawContours(output1, contours, -1, (0,255,0), 3) # image, list of contours, parameter indicating all contour
                                                        #to be drawn, color of contour, thickness is 3 pixel

# create output image and draw bounding box in rectangle
output2 = rgb.copy()

for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)

print(hierarchy[0].shape)
print(contours[0].shape)
print(x, y, w, h)

cv2.rectangle(output2, (x,y), (x+w, y+h), (0,255,0), 2)

# show the images
# cv2.imshow("input", rgb)
# cv2.imshow("mask", red_mask)
# cv2.imshow("output1", output1)
# cv2.imshow("output2", output2)

# cv2.setWindowProperty("input", cv2.WND_PROP_TOPMOST, 1)
# cv2.setWindowProperty("mask", cv2.WND_PROP_TOPMOST, 1)
# cv2.setWindowProperty("output1", cv2.WND_PROP_TOPMOST, 1)
# cv2.setWindowProperty("output2", cv2.WND_PROP_TOPMOST, 1)


cv2.waitKey(0) # waiting for a key pressed
cv2.destroyAllWindows() 

(1, 4)
(140, 1, 2)
43 6 144 72


In [2]:
# color tracking in the video

video = cv2.VideoCapture('Video/City.mp4')

if video.isOpened() == False:
    print('Error opening video')
else:
    fps = video.get(cv2.CAP_PROP_FPS) # or set fps = 30
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            , int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    # print(size)
    cv2.namedWindow('frame')
    cv2.namedWindow('mask')
    cv2.setWindowProperty('frame', cv2.WND_PROP_TOPMOST, 1)
    cv2.setWindowProperty('mask', cv2.WND_PROP_TOPMOST, 1)

    while video.isOpened():
        ret, frame = video.read()
        if ret == True:
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # define the range of "red" color
            # OpenCV HSV scale:
            # H (hue): 0-180
            # S (saturation): 0-255 (0=gray, 255=pure color)
            # V (brightness): 0-255 (0=no brightness, 255=full brightness )

            # mask red color
            red_min = np.array([165, 170, 20])
            red_max = np.array([180, 255, 255])

            # mask blue color
            # red_min = np.array([90, 50, 50])
            # red_max = np.array([130, 255, 255])

            # # mask green color
            # red_min = np.array([40, 50, 50])  # Lower bounds of green color in HSV
            # red_max = np.array([80, 255, 255])  # Upper bounds of green color in HSV

            mask = cv2.inRange(hsv, red_min, red_max)

            # remove small blobs in the mask by morphological operations
            mask = cv2.erode(mask, None, iterations=2)
            mask = cv2.dilate(mask, None, iterations=2)

            # find contour in mask
            contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 3)
            
            cv2.imshow("frame", frame) # shows current frame
            cv2.imshow("mask", mask) # shows current frame 

            time.sleep(1/fps)

            if cv2.waitKey(1)  & 0xFF == ord('x'):
                break
        else:
            break

video.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sudhir/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [20]:
# TO check hsv color  and color tracking in an image

image = cv2.imread("Video/pier.jpg")
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

cv2.imshow("HSV Image", hsv_image)

# Define the color range
red_min = np.array([15,50,180])
red_max = np.array([40,255,255])

# Create a mask for the red color
mask = cv2.inRange(hsv, red_min, red_max)

# Find contours in the mask
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw rectangles around the detected contours
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 3)# smilarly we can do circle mask etc

# Display the original image and the mask
cv2.imshow("Original Image", image)
cv2.imshow("Mask", mask)

cv2.waitKey(0)
cv2.destroyAllWindows()




In [3]:
# color tracking using Cam of Laptop with HSV

# get camera device
cameraCapture = cv2.VideoCapture(0)

# capture the first frame:
success, frame = cameraCapture.read()

# loops untill there is no more frame
while success:
    success, frame = cameraCapture.read() # to get next frame

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    red_min = np.array([165, 150, 50])
    red_max = np.array([180, 255, 255])

    red_mask = cv2.inRange(hsv, red_min, red_max)

    red_mask = cv2.erode(red_mask, None, iterations=3)
    red_mask = cv2.dilate(red_mask, None, iterations=5)

    contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)

    cv2.imshow("Camera", frame)
    cv2.imshow("mask", red_mask)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cameraCapture.release()
cv2.destroyAllWindows()


In [11]:
# constrast brightness : Slow implementation (brute force)

'''
# Point operators:
Basic Linear Function: g(x) = alpha x f(x) + beta --> g(i,j) = alpha x f(i.j) + beta
alpha --> gain --> contrast
beta --> bias --> brightness

g(x) = (1-alpha) x f0(x) + alpha x f1(x)
dst = alpha x src1 + beta x src2 + gamma
'''

def constrastBrightnessSlow(input,  output, alpha, beta):
    # Uisng loop
    for y in range (input.shape[0]): # height
        for x in range (input.shape[1]): # width
            if (len(input.shape)==2):
                output[y,x] = np.clip(alpha*input[y,x] + beta, 0, 255)
            if (len(input.shape)==3):
                output[y,x,:] = np.clip(alpha*input[y,x,:] + beta, 0, 255)

# read image
input = cv2.imread("Video/pier.jpg", 0)

# initialize alpha (contrast) and beta (brightness)
alpha = 2.0
beta = 40

# create output matrix and initial pixel value equal to zero , with same size and type of original image
output = np.zeros(input.shape, input.dtype)

constrastBrightnessSlow(input, output, alpha, beta)

# create window and show image
cv2.namedWindow('Contrast and brigthness', 1)
cv2.imshow("Contrast and Brightness", output)

cv2.waitKey(0) # waiting for a key pressed
cv2.destroyAllWindows()




In [22]:
# optimized code:

def contrastBrigtness(input, output, alpha, beta):
    # using numpy slicing
    if (len(input.shape)==2):
        height, width = input.shape
        print(height, width)
        output[0:height, 0:width] = np.clip(alpha * input[0:height,0:width]+beta, 0, 255)

    if (len(input.shape)==3):
        height, width, channel = input.shape
        output[:height, :width, :channel] = np.clip(alpha*input[:height, :width, :channel]+beta, 0, 255)
    
input = cv2.imread('Video/pier.jpg')
input[:,:,1] = 0
my_roi = input[0:100, 0:100]
input[300:400, 300:400] = my_roi

cv2.namedWindow('Constract and brighness', 1)

alpha = 1.0
beta = 0.0

while(True):

    output = np.zeros(input.shape, input.dtype)

    contrastBrigtness(input, output, alpha, beta)

    # put text into the input image
    cv2.putText(output, 'aplha: '+ str(format(alpha, '.1f')) + 'beta: '+str(format(beta, '.2f')), (25, 40), cv2.FONT_HERSHEY_PLAIN, 2.0, (255, 255, 255), 2)
    cv2.putText(output, '[ESC] quit', (output.shape[1]-120, 30), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 1)

    cv2.imshow('constrast and brightness', output)

    # handle keyboard event
    key = cv2.waitKey(20) # wait 10 ms and get keyboard event

    if key == 119: # w : increase contrast
        if(alpha < 3.0):
            alpha += 0.1
    elif key == 121 : # y : decrease contrast
        if(alpha > 0.2):
            alpha -= 0.1
    elif key == 115: # s : increase brightness
        if(beta < 100):
            beta += 1
    elif key == 97: # a
        if(beta > 0):
            beta -= 1
    elif key == 27: # Esc
        break

cv2.destroyAllWindows()



In [18]:
# using addweighted function to change brightness and contrast


input = cv2.imread('Video/pier.jpg', 1)

cv2.namedWindow('Contrast and brightness', 1)

alpha = 1.0
beta = 0.0

while True:
    output = np.zeros(input.shape, input.dtype)
    output = cv2.addWeighted(input, alpha, input, 0, beta, output)

    # put text
    cv2.putText(output, 'alpha: '+ str(format(alpha, '.1f'))+ " beta : "+ str(format(beta, '.1f')), (25, 40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (255, 255, 2555), 2)
    cv2.putText(output, '[ESC] : quit', (output.shape[1]-120, 30), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 1)

    cv2.imshow('contrast and brightness', output)

    key = cv2.waitKey(10)
    if key == 119: # w => increase contrast
        if(alpha < 3.0): 
            alpha += 0.1
    elif  key == 121: # y => decrease contrast
        if(alpha > 0.2): 
            alpha -= 0.1
    elif  key == 115: # s => increase brightness
        if(beta < 100): 
            beta += 1
    elif  key == 97: # a => decrease brightness
        if(beta > 0): 
            beta -= 1
    elif  key == 27: # ESC => break the loop
        break

cv2.destroyAllWindows()


In [21]:
# image blending

# read image
img1 = cv2.imread('Video/pier.jpg', 1)
img2 = cv2.imread('Video/beach.jpg', 1)

# create window
cv2.namedWindow('Image blending', 1)

# initialize alpha(contract)
alpha = 0.0

while True:

    # create output matrix and initialize pixel values
    output = np.zeros(input.shape, input.dtype)

    # initialize beta(brightness)
    beta = 1.0 - alpha

    # Use addWeighted function to adjust contrast and brightness
    cv2.addWeighted(img1, alpha, img2, beta, 0, output)

    # put text into the image
    cv2.putText(output, "alpha: "+str(format(alpha, '.1f')), (25, 40), cv2.FONT_HERSHEY_PLAIN, 2.0, (255,255,255), 2)
    cv2.putText(output, "[ESC] quit", (output.shape[1]-120, 30), cv2.FONT_HERSHEY_PLAIN, 1.0, (255,255,255), 1)

    # show image
    cv2.imshow('Image blend', output)

    # handle keyboard event
    key = cv2.waitKey(20)

    if key == 115 : #s: increase alpha
        if (alpha < 0.9):
            alpha += 0.1
    elif key == 97: # a : decrease alpha
        if (alpha > 0.1):
            alpha -= 0.1
    elif key == 27: # [ESC] quit
        break

cv2.destroyAllWindows()

In [ ]:
# Example of passing argument 

# Create an ArgumentParser object
# parser = argparse.ArgumentParser(description='My Command-Line Program')

# # Add arguments
# parser.add_argument('input_file', help='Path to the input file')
# parser.add_argument('-o', '--output', help='Path to the output file')

# # Parse the command-line arguments
# args = parser.parse_args()

# # Access the values of the arguments
# input_file = args.input_file
# output_file = args.output

# # Use the values in your program
# print("Input file:", input_file)
# print("Output file:", output_file)
